In [1]:
import random
import numpy as np
import cv2
import os
import math
import time
import random
import matplotlib.pyplot as plt
import sys

In [2]:
# height = 19
# width = 19
# W = 361
height = 15
width = 15
W = 225
# W = 212
b = 32
k = 40
step = 0.1
times  = 10
tree_z = list()
tree_sita = list()


In [3]:
class Node(object):
    def __init__(self, S):
        self.W_S, self.N_S = S.shape
        self.S = S
        seed = time.time()
        self.leftnode = None
        self.rightnode = None
        self.prefect_z = None
        self.prefect_sita = None
        self.prefect_right_S = None
        self.prefect_left_S = None
        np.random.seed(int(time.time()))
       
    
    def find_the_biggest_information_gain(self):
        
        HS = self.entropy(self.S)
        # print('当前节点的总熵为' + str(HS))
        info_gain =-999.0
        
        for i in range(times):
#             start = time.time()
            if i % 5 == 0:
                print("当前运行到%d" % (i))
            z = np.random.rand(W, 1) * 2 -1
            zero_index = np.random.rand(len(z) - k) * len(z)
            for z_i in zero_index:
                z[math.floor(z_i)] = 0.0
#             sita = np.random.rand(self.N_S,1) * 2 -1
            sita = np.random.rand()*2-1
            result = np.dot(self.S.T, z) + sita
            cnt_P = 0
            cnt_N = 0
            P_index = []
            N_index = []
            
#             P_index = [ii for ii in range(self.N_S) if result[ii][0] >= 0]
#             cnt_P = len(P_index)
#             N_index = [ii for ii in range(self.N_S) if result[ii][0] < 0]
#             cnt_N = len(N_index)

            for ii in range(self.N_S):
                if result[ii][0] >= 0:
                    P_index.append(ii)
#                     cnt_P += 1
                else:
                    N_index.append(ii)
#                     cnt_N += 1
            cnt_N = len(N_index)
            cnt_P = len(P_index)
            pos_S = self.S[:, P_index]
            neg_S = self.S[:, N_index]
#             if cnt_N == 0 or cnt_P ==0:
#                 continue
#             print('进行了一次分类，然后将分类结果求解信息增益' + str(neg_S.shape) +str(pos_S.shape))
            ans = self.information_gain(cnt_P, cnt_N, pos_S, neg_S)
            mid_v = HS - ans 
#             print(mid_v)
            if mid_v > info_gain: 
#             if mid_v < info_gain: 
                info_gain = mid_v
                self.prefect_z = z
                self.prefect_sita = sita
                self.prefect_right_S  = pos_S
                self.prefect_left_S = neg_S
#             end = time.time()
#             print('共用时%d' % (end - start))
        tree_sita.append(self.prefect_sita)
        tree_z.append(self.prefect_z)
        
        print('已经完成150次随机尝试，选出了当前最好效果的z和sita')
        if self.prefect_right_S.shape[1] and self.prefect_left_S.shape[1]:
            new_node_right = Node(self.prefect_right_S)
            print('建立右节点 ' + '右节点数据为' + str(self.prefect_right_S.shape))
            self.rightnode = new_node_right
            new_node_left = Node(self.prefect_left_S)
            self.leftnode = new_node_left
            print('建立左节点 ' + '左节点数据为' + str(self.prefect_left_S.shape))
            
    
    def information_gain(self, cnt_P, cnt_N, pos_S, neg_S):
        try:
            return float(cnt_N / self.N_S) *self.entropy(neg_S) + float(cnt_P / self.N_S) * self.entropy(pos_S) 
        except:
            print ('该节点的总样本数目为' + str(self.S))
           
    def entropy(self, s):
#         print('要进行协方差运算的shape' + str(s.shape))
        con_arr = np.cov(s)
#         print('协方差运算结束之后的shape' + str(con_arr.shape))
        det = float(np.linalg.det(con_arr)) 
        # print('行列式的值为' + str(det))
#         if det <= 0:
#             det = 0.1 ** 6
        try:
            Hs = 0.5 * W* math.log(2 * math.pi * math.e) + 0.5 * math.log(det)
        except:
            print('det is' + str(det))
            Hs = 0.5 * W* math.log(2 * math.pi * math.e) + 0.5 # 将det 设置增大 然后把这个结果去掉（当前还有隐患）
#         if np.isnan(Hs):
#             Hs = 0
#         assert Hs >= 0
        return Hs  
    


In [4]:
def build_tree(root, height):
    if height == 5:
        return
    # print(root.S)
    root.find_the_biggest_information_gain()
    
    if root.leftnode != None and root.rightnode != None:
        build_tree(root.leftnode, height + 1)
        build_tree(root.rightnode, height + 1)
        

In [5]:
# def travel_tree(root):
#     if root.leftnode!=None:
#         travel_tree(root.leftnode)
#     print(root.prefect_z)
#     if root.rightnode!=None:
#         travel_tree(root.rightnode)

In [269]:
# x_y_change = list()
# change_mode = [-1, 0, 1]
# for i in change_mode:
#     for ii in change_mode:
#         x_y_change.append((i, ii))
        
# x_y_change
# def change_position_point(p):
#     after_change = list()
#     for i in x_y_change:
#         after_change.append((p[0] + i[0], p[1] + i[1]))
#     return after_change

采用CMN方法选取组合，该部分主要用于对一个patch中多个点进行选取

In [270]:
#  def C_M_N(m,n, time, st, v, choice_mix_cmn,match):
#         if time == 0:
#             for i in range(m):
#                 new_st = ''
#                 new_st += str(i)
#                 C_M_N(m,n,time+1,new_st,i, choice_mix_cmn,match)
#         elif time == n:
#     #         print(st)
#             choice_mix_cmn.append(st)
#         else:
#             for i in match[v]:
#                 new_st = st
#                 new_st += str(i)
#                 C_M_N(m,n,time+1,new_st,i, choice_mix_cmn,match)
                
# def C_M_N_ALL(m,n):
#     choice_mix_cmn = list()
#     match = dict()
#     for i in range(6):
#         match[i] = range(i+1,6)
#     C_M_N(m,n,0,'',0,choice_mix_cmn,match)
#     return choice_mix_cmn

采用AMN方法选取组合，该部分主要用于对一个patch中多个点进行扰动

In [271]:
# def A_M_N(m,n, time, st, v,choice_mix_amn,match_):
#         if time == 0:
#             for i in range(m):
#                 new_st = ''
#                 new_st += str(i)
#                 A_M_N(m,n,time+1,new_st,i,choice_mix_amn,match_)
#         elif time == n:
#             choice_mix_amn.append(st)
          
#         else:
#             for i in match_[v]:
#                 new_st = st
#                 new_st += str(i)
#                 A_M_N(m,n,time+1,new_st,i,choice_mix_amn,match_)
                
# def A_M_N_ALL(m,n):
#     choice_mix_amn = list()
#     match_ = dict()
#     for i in range(9):
#         match_[i] = range(9)
#     A_M_N(m,n,0,'',0,choice_mix_amn,match_)
#     return choice_mix_amn

In [272]:
# A_M_N_ALL(9,2)

In [273]:
def shake_position(points, shake):
    tem = list()
    cnt = 0
    for  p in points:
#         entry_0 = p[0] + math.floor(shake[cnt])
        entry_0 = p[0]
        cnt += 1
        entry_1 = p[1] + math.floor(shake[cnt])
        cnt += 1
        tem.append((entry_0, entry_1))
    return tem

In [274]:
image_template = np.zeros((19, 19), np.float32)
cv2.circle(image_template, (9,9), 7, (1,), -1)
# cv2.imshow('show', image_template)
print(image_template)
image_template = image_template.reshape(19*19)
index_pick = [x for x in range(19*19) if image_template[x] == 0]
print(len(index_pick))
print(image_template[index_pick])


[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.]
 [ 0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.
   0.]
 [ 0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.
   0.]
 [ 0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.
   0.]
 [ 0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.
   0.]
 [ 0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.
   0.]
 [ 0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.
   0.]
 [ 0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.
   0.]
 [ 0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.
   0.]
 [ 0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.
   0.]
 [ 0.  0.  0.  1.  1.  1.  1

In [275]:
def generate(width, height, num, gap, seed):
    random.seed(seed)
    
    point_index = np.arange(width * height)
    unused_point_set = np.arange(width * height)
    unused_point_num = width * height

    used_point_set = []
    used_point_num = 0
    while used_point_num != num and unused_point_num != 0:
        rd = random.randint(0, unused_point_num - 1)
        x = unused_point_set[rd] % width
        y = int(unused_point_set[rd] / width)
        used_point_set.append((x, y))
        used_point_num += 1
        point = unused_point_set[rd]
        unused_point_set[rd] = unused_point_set[unused_point_num - 1]
        point_index[unused_point_set[unused_point_num - 1]] = rd
        point_index[point] = -1
        unused_point_num -= 1


        for i in range(int(x - gap), int(x + gap + 1)):
            h = int(math.floor((gap ** 2 - (x - i) ** 2) ** .5))
            ii = i % width
            for j in range(int(y - h), int(y + h + 1)):
                jj = j % height
                index = jj * width + ii
                if point_index[index] >= 0:
                    unused_point_set[point_index[index]] = unused_point_set[unused_point_num - 1]
                    point_index[unused_point_set[unused_point_num - 1]] = point_index[index]
                    point_index[index] = -1
                    unused_point_num -= 1

    return used_point_set

def generate_2(width, height, num, gap, seed):
    
    random.seed(seed)
    
    point_index = np.arange(width * height)
    unused_point_set = np.arange(width * height)
    unused_point_num = width * height

    margin_index = np.zeros((width * height), np.int32) - 1
    margin_set = np.zeros((width * height), np.int32)
    margin_num = 0

    used_point_set = []
    used_point_num = 0

    rd = random.randint(0, unused_point_num - 1)
    margin_set[margin_num] = unused_point_set[rd]
    margin_index[unused_point_set[rd]] = margin_num
    margin_num += 1
    point = unused_point_set[rd]
    unused_point_set[rd] = unused_point_set[unused_point_num - 1]
    point_index[unused_point_set[unused_point_num - 1]] = rd
    point_index[point] = -1
    unused_point_num -= 1

    while used_point_num != num and margin_num != 0:
        rd = random.randint(0, margin_num - 1)
        x = margin_set[rd] % width
        y = margin_set[rd] / width
        used_point_set.append((x, y))
        used_point_num += 1

        point = margin_set[rd]
        margin_set[rd] = margin_set[margin_num - 1]
        margin_index[margin_set[margin_num - 1]] = rd
        margin_index[point] = -1
        margin_num -= 1

        gap_out = gap + 1

        for i in range(x - int(math.ceil(gap_out - 1)), x + int(math.ceil(gap_out - 1)) + 1):
            h_out = int(math.ceil((gap_out ** 2 - (x - i) ** 2) ** .5 - 1))
            h_in = int(math.ceil((gap ** 2 - (x - i) ** 2) ** .5 - 1))
            ii = i % width
            for j in range(y - h_out, y + h_out + 1):
                jj = j % height
                index = jj * width + ii

                is_margin = False
                if j < y - h_in or j > y + h_in:
                    is_margin = True

                if is_margin:
                    if point_index[index] >= 0:
                        margin_set[margin_num] = index
                        margin_index[index] = margin_num
                        margin_num += 1


                else:
                    if margin_index[index] >= 0:
                        margin_set[margin_index[index]] = margin_set[margin_num - 1]
                        margin_index[margin_set[margin_num - 1]] = margin_index[index]
                        margin_index[index] = -1
                        margin_num -= 1

                if point_index[index] >= 0:
                    unused_point_set[point_index[index]] = unused_point_set[unused_point_num - 1]
                    point_index[unused_point_set[unused_point_num - 1]] = point_index[index]
                    point_index[index] = -1
                    unused_point_num -= 1

    return used_point_set

def generate_patch():

    dropout_ratio = [.0, .1, .2, .4]

    width = 120
    height = 80
    gap = 7
    point_num = 250

    point_radius = 2

    shift = 1

    offset = 20


    
    
    scale_factor = 4
    points = generate(width*scale_factor, height*scale_factor, point_num, int(gap*scale_factor-scale_factor/4*1), 8888)
#     print(points)

#     display = np.zeros((height*scale_factor, width*scale_factor), np.float32)
#     for x, y in points:
#         cv2.circle(display, (x, y), int(point_radius*scale_factor*.6), (1,), -1)
#     cv2.imshow('hehe', display)
#     cv2.waitKey()
#     train_x= list()
#     for x, y in points:
#         # print(display[x-6:x+5,y-6:y+5].shape)
#         try: 
#             train_x.append(display[x-6:x+5,y-6:y+5].reshape(121))
#         except:
#             pass
# #     print(len(train_x))
#     train_ = np.array(train_x).reshape(len(train_x), 121).T
#     print(train_.shape)
# #     print(train_)
#     return train_
    
   
    train_x= list()
    points = [(int(round(x/float(scale_factor))), int(round(y/float(scale_factor)))) for x, y in points]
    p_equal_height = dict()
    for index, p in enumerate(points):
        tem = [x for x in range(len(points)) if points[x][0] >= (p[0]- 3) and points[x][0] <= (p[0] + 2)   ]
        print(len(tem))
        p_equal_height[index] = tem
    print(p_equal_height)
    print('原始点的数目为' + str(len(points)))
    print(points)
    for counter in range(1000):
        image = np.zeros((height, width), np.float32)
        image_fill = np.zeros((height + offset*2, width + offset*2), np.float32)
        random.seed(time.time())
        shake = np.random.rand(len(points)*2) * 3 - 1
#         print('shake is ' + str(shake))
        new_points = shake_position(points, shake)
    #     print('the shape of img_fill is '+ str(image_fill.shape))
        print('抖动之后的点的数目为' + str(len(new_points)))
        for p in new_points:

            cv2.circle(image, p, point_radius, (1,), -1)
            x = p[0] + offset
            y = p[1] + offset
    #         print('x is %d, y is %d' % (x,y))
            cv2.circle(image_fill, (x, y), point_radius, (1,), -1)
            if x >= width:
                cv2.circle(image_fill, (x-width, y), point_radius, (1,), -1)
            if x < offset*2:
                cv2.circle(image_fill, (x+width, y), point_radius, (1,), -1)
            if y >= height:
                cv2.circle(image_fill, (x, y-height), point_radius, (1,), -1)
            if y < offset*2:
                cv2.circle(image_fill, (x, y+height), point_radius, (1,), -1)
            if x >= width and y >= height:
                cv2.circle(image_fill, (x-width, y-height), point_radius, (1,), -1)
            if x < offset*2 and y < offset*2:
                cv2.circle(image_fill, (x+width, y+height), point_radius, (1,), -1)
            if y >= height and x < offset*2:
                cv2.circle(image_fill, (x+width, y-height), point_radius, (1,), -1)
            if y < offset*2 and x >= width:
                cv2.circle(image_fill, (x-width, y+height), point_radius, (1,), -1)



#         cv2.rectangle(image_fill, (offset, offset), (offset + width, offset + height), (64,), 1)
#         cv2.imshow('hehe', image_fill)
#         cv2.waitKey()

        for p in new_points:   
            x = p[0] + offset
            y = p[1] + offset
            left_b = x-9
            right_b = x+10
            up_b = y-9
            down_b = y+10
#             print(left_b, right_b, up_b, down_b)
#             print(image.shape)
#             print(image_fill[up_b:down_b, left_b:right_b].shape)
            train_x.append(image_fill[up_b:down_b, left_b:right_b].reshape(19*19)[index_pick])
    print(len(train_x))
    train_ = np.array(train_x).reshape(len(train_x), 212).T
    return train_, p_equal_height

#     points_extend = [(p[0] + offset, p[1] + offset) for p in points]
#     print('点数'+ str(len(points_extend)))
#     train_x= list()
#     for p in points_extend:   
# #         try: 
#         left_b = x-9
#         right_b = x+10
#         up_b = y-9
#         down_b = y+10
#         loc_in_patch = [po for po in points_extend if po[0] in range(left_b, right_b) and po[1]in range(up_b, down_b)]
#         # print('在该范围内的点数' + str(len(loc_in_patch)))
        # 该部分是对其中一个点进行扰动的代码
#         for pp in loc_in_patch:
#             image_fill[pp[0],pp[1]] = 0
#             after_change_pos = change_position_point(pp)
# #             print('改变行动模式后的方案'+ str(len(after_change_pos)))
# #             print(after_change_pos)
#             for p_ in after_change_pos:
#                 if image_fill[p_[0],p_[1]] == 1:
#                     change_flag = False 
#                 else:
#                     change_flag = True
#                     image_fill[p_[0],p_[1]] = 1
#                 # print(image_fill[left_b:right_b,up_b:down_b].shape)
#                 train_x.append(image_fill[left_b:right_b,up_b:down_b].reshape(19*19))    
#                 if change_flag == True:
#                     image_fill[p_[0],p_[1]] = 0
# #         except:
# #             pass
#             image_fill[pp[0],pp[1]] = 1
#         for n_choice in n_choice_list:
#             choice_plan =  C_M_N_ALL(len(loc_in_patch),n_choice)
#             for plan in choice_plan:
#                 change_patch = list()
#                 assert type(plan) is str
#                 for choice_ch in plan:
#                     choice_int = int(choice_ch)
#                     change_patch.append(loc_in_patch[choice_int])
# #                 print('需要改的范围为')
# #                 print(change_patch)
#                     AMN_plan = list()
#                     for pp in change_patch:
#                         after_change_pos = change_position_point(pp)
#                         AMN_plan.append(after_change_pos)
#                     choice_plan_AMN = A_M_N_ALL(len(AMN_plan[0]), n_choice)
#                     for st in choice_plan_AMN:
#                         assert type(st) is str
#                         plan = [int(c) for c in st]
#                         change_flag_list = list()
#                         for key, AMN_p in enumerate(AMN_plan):
#                             image_fill[AMN_p[5][0],AMN_p[5][1]] = 0
#                             if image_fill[AMN_p[plan[key]][0], AMN_p[plan[key]][1]] ==1 :
#                                 change_flag = False 
#                             else:
#                                 change_flag = True
#                                 image_fill[AMN_p[plan[key]][0], AMN_p[plan[key]][1]] =1
#                             change_flag_list.append(change_flag)
#                         if len(train_x) <=900000:    
#                             train_x.append(image_fill[left_b:right_b,up_b:down_b].reshape(19*19)) 
#                         for key, AMN_p in enumerate(AMN_plan):
#                             if change_flag_list[key] == True:
#                                 image_fill[AMN_p[plan[key]][0], AMN_p[plan[key]][1]] =0
#                             image_fill[AMN_p[5][0],AMN_p[5][1]] = 1
  
#     print(len(train_x))
# #     train_ = np.array(train_x).reshape(len(train_x), 19*19).T
#     return train_
        
        


In [276]:
test_S, p_equal_height = generate_patch()

7
9
8
6
6
8
10
9
6
8
7
7
8
9
9
6
8
8
7
8
8
8
8
7
6
9
8
9
8
8
9
7
8
7
9
7
8
8
9
8
8
8
8
7
7
6
8
8
6
10
6
7
7
8
9
7
9
9
6
6
7
9
8
7
6
8
7
9
4
6
7
6
8
4
8
7
9
8
8
7
10
6
8
7
8
8
9
9
8
9
7
8
7
9
8
9
7
8
9
8
8
8
9
9
7
8
8
7
5
7
7
6
6
7
8
9
6
8
8
8
8
9
9
4
8
9
6
9
10
8
7
8
7
9
9
7
9
8
7
7
6
10
6
7
6
5
10
7
10
{0: [0, 30, 64, 86, 109, 116, 122], 1: [0, 1, 30, 81, 86, 109, 115, 122, 127], 2: [2, 35, 36, 65, 110, 117, 124, 130], 3: [3, 27, 50, 59, 92, 111], 4: [4, 43, 45, 47, 48, 140], 5: [5, 16, 21, 42, 78, 102, 118, 120], 6: [6, 12, 31, 84, 93, 98, 100, 136, 139, 146], 7: [7, 8, 25, 58, 61, 67, 95, 134, 145], 8: [7, 8, 58, 61, 143, 145], 9: [9, 19, 47, 72, 101, 129, 135, 137], 10: [10, 18, 29, 80, 94, 107, 128], 11: [11, 44, 66, 83, 92, 132, 147], 12: [6, 12, 31, 32, 84, 100, 139, 146], 13: [13, 14, 22, 33, 40, 51, 76, 97, 126], 14: [13, 14, 22, 33, 40, 51, 76, 97, 126], 15: [15, 68, 73, 77, 123, 131], 16: [5, 16, 21, 42, 78, 108, 118, 120], 17: [15, 17, 54, 56, 77, 106, 121, 131], 18: [18, 2

抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点

抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点的数目为149
抖动之后的点

In [17]:
import cv2
import os
import numpy as np
import time
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.tri as tri
% matplotlib inline

photo = cv2.imread(os.path.join('photo', '0l.png'), 0)
# min_v = 999
# max_v = -999
# for line in photo:
#     if min(line) < min_v:
#         min_v = min(line)
#     if max(line) > max_v:
#         max_v = max(line)
# print(max_v, min_v)

# photo = (photo - min_v)/(max_v - min_v)
# print(photo)
mu = np.average(photo)
sigma = np.std(photo)
print(mu, sigma)
photo = float((photo - mu) / sigma)
print(photo)
def generate_patch():
    width = 15
    height = 15
    train_x = list()
    boundary_w = 0
    boundary_h = 0
    boundary_h, boundary_w = photo.shape
    for i in range(boundary_w):
        for ii in range(boundary_h):
            if i + width < boundary_w and ii + height < boundary_h:
                train_x.append(photo[ ii:ii+height, i:i+width].reshape(width*height))
    print(len(train_x))
    train_ = np.array(train_x).reshape(len(train_x), width * height).T
    return train_

        

test_S = generate_patch()
# mu = np.average(test_S)
# sigma = np.std(test_S)
# print(mu, sigma)
# test_S = (test_S - mu) / sigma


25.0094199219 35.9819825672
[[-0.36155373 -0.36155373 -0.36155373 ..., -0.36155373 -0.36155373
  -0.36155373]
 [-0.36155373 -0.36155373 -0.36155373 ..., -0.36155373 -0.36155373
  -0.36155373]
 [-0.36155373 -0.36155373 -0.36155373 ..., -0.36155373 -0.36155373
  -0.36155373]
 ..., 
 [-0.36155373 -0.36155373 -0.36155373 ..., -0.36155373 -0.36155373
  -0.36155373]
 [-0.36155373 -0.36155373 -0.36155373 ..., -0.36155373 -0.36155373
  -0.36155373]
 [-0.36155373 -0.36155373 -0.36155373 ..., -0.36155373 -0.36155373
  -0.36155373]]
993025


In [14]:
del test_S

print(test_S.shape)

NameError: name 'test_S' is not defined

In [18]:
print(test_S.shape)
root = Node(test_S)
build_tree(root, 0)
# travel_tree(root)

(225, 993025)
det is0.0
当前运行到0
该节点的总样本数目为[[-0.36155373 -0.36155373 -0.36155373 ..., -0.38934541 -0.38934541
  -0.36155373]
 [-0.36155373 -0.36155373 -0.36155373 ..., -0.38934541 -0.38934541
  -0.36155373]
 [-0.36155373 -0.36155373 -0.36155373 ..., -0.38934541 -0.38934541
  -0.36155373]
 ..., 
 [-0.36155373 -0.36155373 -0.36155373 ..., -0.36155373 -0.36155373
  -0.36155373]
 [-0.36155373 -0.36155373 -0.36155373 ..., -0.36155373 -0.36155373
  -0.36155373]
 [-0.36155373 -0.36155373 -0.36155373 ..., -0.36155373 -0.36155373
  -0.36155373]]


TypeError: unsupported operand type(s) for -: 'float' and 'NoneType'

In [278]:
all_z = np.array(tree_z).reshape(len(tree_z), 212).T
all_sita = np.array(tree_sita)
print(all_sita.shape)
result = np.dot(test_S.T, all_z) + all_sita


(31,)


In [279]:
def array_0_1_change(array):
    x, y = array.shape
    for i in  range(x):
        for ii in range(y):
            if array[i][ii] >=0:
                array[i][ii] = 1
            else:
                array[i][ii] = 0
    return array

In [280]:
def distance_hamming(a,  b):
    len_a= a.shape
    cnt = 0
    for i in range(len_a[0]):
        if a[i] != b[i]:
            cnt += 1
    return cnt

In [281]:
result_ = array_0_1_change(result)
print(result_.shape)

    

(149000, 31)


In [282]:
# bound = 11
# plus = 0
# cnt = 0
# for ii in range(10):
#     for i in range(149):
#         plus += distance_hamming(result_[ii * 149], result_[i + ii * 149])
#         if distance_hamming(result_[0], result_[i]) <= bound:
#             cnt += 1
#             # print(distance_hamming(result_[0], result_[i]))
# print (plus/ 1490)
# print((cnt -10)/ 1490)
# FP = cnt -10
# FF = 1480 - cnt

    

# plus_ = 0
# cnt_ = 0
# for ii in range(149):
#     for i in range(10):
#         plus_ += distance_hamming(result_[ii], result_[i* 149 + ii])
#         if distance_hamming(result_[ii], result_[i* 149 + ii]) <= bound:
#             cnt_ += 1
#        #  print(distance_hamming(result_[0], result_[i* 149]))
# print(plus_/1490)
# print(cnt_ /1490)
# PP =  cnt_
# PN = 1490 - cnt_
# precision = PP/(FP +  PP)
# recall = (PP)/(PP+ FF)
# f1 = (precision + recall)/2
# print('准确率是%f,召回率是%f,f1是%f'%(precision, recall, f1))


In [283]:
# bound = 11
# plus = 0
# cnt = 0
# cnt_is = 0
# cnt_not = 0
# for ii in range(1, 10):
#     for i in range(149):
#         plus += distance_hamming(result_[i], result_[i + ii * 149])
#         cnt += 1
#         if distance_hamming(result_[i], result_[i + ii * 149])< bound:
#             cnt_is += 1
#         # print(distance_hamming(result_[i], result_[i + ii * 149]))
# print('同类间的平均距离为 ' + str(plus/cnt))
# a = cnt_is/cnt
# print('预测正确率 ' + str(cnt_is/cnt))
plus = 0
cnt = 0
min_pos = 0

for i in range(149):
    for ii in range(1000):
        min_d = 9999
        min_pos = -1
        for iii in p_equal_height[i]:
            tem_d = distance_hamming(result_[i], result_[iii + ii * 149])
            if tem_d < min_d:
                min_d  = tem_d
                min_pos = iii
        if min_pos == i:
            cnt += 1
print(float(cnt /149000))
        
        
        
        
#             cnt += 1
#             if  < bound:
#                 cnt_not += 1
#             # print(distance_hamming(result_[i], result_[iii + ii * 149]))
# print('类间距离为' +  str(plus/cnt))
# print('预测错误率' + str(cnt_not / cnt))
# b = cnt_not / cnt
# print('正确率是'+ str(a/(a+b)))

0.6619395973154363


<p>随机数设置为 10次的时候: 正确率为51.489%.<p>
<p>随机数设置为50时， 正确率为57.079%<p>
<p> 随机数为 100时，正确率为57.215%<p>
<p> 随机数为 500时，正确率为65.165%<p>  
<p> 随机数为 100时，正确率为65.165%<p>

添加八种行为模式

添加扰动方针：
    首先找取图片当中的光斑位置
    在光斑位置处画圆
    遍历光斑位置
        以该点为中心，取19*19区域
            如果存在意外 则下移或补全
        找取光斑中心在该光源中的数目：
            按照行为模式添加扰动
            将所有扰动全部添加到样本集合当中
    查看当前的样本维度，保证其数量为十万到百万级别

将训练样本放入到决策树当中：
    无监督训练树的节点
    然后将每个非叶子节点的z和 sita值保存下来（该操作在遍历树节点的时候进行，此处可以选择 层次遍历，比较符合我们的行为模范）
    将训练集.T * z + sita 的结果保存下来，然后寻找相应的汉明距离， 汉明距离小于等于5的 可以归成一类

In [222]:
import numpy as np
import os
import cv2

In [223]:
cap = cv2.VideoCapture(os.'vtest.avi')
while(cap.isOpened()):
    ret, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cv.imshow('frame',gray)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()

SyntaxError: invalid syntax (<ipython-input-223-52c954d4cb39>, line 1)

In [5]:
import numpy as np

a = np.array([[1,1,1], [2,2,2]])
a.T.shape                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
np.linalg.det(np.cov(a.T))

0.0